## TASK 1

In [165]:
class environment:
    
    def __init__(self, location, A_state, B_state, steps):
        self.agent_loc = location
        self.A_state = A_state
        self.B_state = B_state
        self.action = ''
        self.steps = steps


    def display(self):
        print(f"A={self.A_state}, B={self.B_state}")
        print(f"Agent Location : {self.agent_loc}")

   
    def determine_action(self):

        if self.agent_loc == 'A':
            
            # if clean, move; else suck
            if self.A_state.upper() == 'CLEAN':
                self.action = 'MOVE_RIGHT'

            elif self.A_state.upper() == 'DIRTY':
                self.action = 'SUCK'

        elif self.agent_loc == 'B':
            
            # if clean, move; else suck
            if self.B_state.upper() == 'CLEAN':
                self.action = 'MOVE_LEFT'

            elif self.B_state.upper() == 'DIRTY':
                self.action = 'SUCK'


    def action_done(self):

        while self.steps > 0:

            self.determine_action()

            print(f"A={self.A_state}, B={self.B_state}")
            print(f"Agent Location : {self.agent_loc}   Action : {self.action}")

            if self.agent_loc == 'A' and self.A_state == 'DIRTY':
                self.A_state = 'CLEAN'
                self.agent_loc = 'B'

            elif self.agent_loc == 'B' and self.B_state == 'DIRTY':
                self.A_state = 'CLEAN'
                self.B_state = 'A'

            elif self.agent_loc == 'A':
                self.agent_loc = 'B'
            elif self.agent_loc == 'B':
                self.agent_loc = 'A'

            self.steps -= 1


In [167]:
env1 = environment('A', 'DIRTY', 'CLEAN', 5)
env1.action_done()

A=DIRTY, B=CLEAN
Agent Location : A   Action : SUCK
A=CLEAN, B=CLEAN
Agent Location : B   Action : MOVE_LEFT
A=CLEAN, B=CLEAN
Agent Location : A   Action : MOVE_RIGHT
A=CLEAN, B=CLEAN
Agent Location : B   Action : MOVE_LEFT
A=CLEAN, B=CLEAN
Agent Location : A   Action : MOVE_RIGHT


## TASK 2

In [158]:
import math
import numpy as np
import random


def flatten_list(l):

    res = []

    for row in l:
        res.append(row)

    return res


class environment:

    def __init__(self, m, n, dirt_rate=0.2, wall_rate=0.1):
        self.m = m
        self.n = n
        self.dirt_rate = dirt_rate
        self.wall_rate = wall_rate
        self.grid = []
        self.dirt_count = math.ceil(self.m * self.n * self.dirt_rate)
        self.wall_count = math.ceil(self.m * self.n * self.wall_rate)
        self.agent_i = -1
        self.agent_j = -1
        self.reward = 100

    
    def create_environment(self):

        
        for _ in range(self.m):

            temp = []

            for _ in range(self.n):
                temp.append('-')

            self.grid.append(temp)

        total_dirt = math.ceil(self.m * self.n * self.dirt_rate)
        total_wall = math.ceil(self.m * self.n * self.wall_rate)
        dirt_locs = []
        wall_locs = []


        while total_dirt > 0:
            new_num = np.random.randint(0, (self.m*self.n))
            if new_num not in dirt_locs:
                dirt_locs.append(new_num)
                total_dirt -= 1

        while total_wall > 0:
            new_num = np.random.randint(0, (self.m*self.n))
            if new_num not in wall_locs and new_num not in dirt_locs:
                wall_locs.append(new_num)
                total_wall -= 1

        temp_grid = np.asarray(self.grid)
        flattened_grid = temp_grid.flatten()

        for loc in dirt_locs:
            flattened_grid[loc] = 'D'

        for loc in wall_locs:
            flattened_grid[loc] = 'W'

        self.grid = flattened_grid.reshape(self.m, self.n)

        for i in range(self.m):
            for j in range(self.n):

                if self.grid[i, j] == '-':
                    self.grid[i, j] = 'A'
                    self.agent_i = i
                    self.agent_j = j
                    break
            
            if self.grid[i, j] == 'A':
                break

   
    def display_environment(self):

        for i in range(self.m):

            print('| ', end='')
            for j in range(self.n):

                print(self.grid[i][j], end='   ')
            
            print('| \n')
        
        print()

    
    def get_new_pos(self, i, j):

        moves = [ (0, 1), (1, 0), (-1, 0), (0, -1) ]

        while True:

            chosen_move = random.choice(moves)

            new_pos = ( i - chosen_move[0], j - chosen_move[1] )

            if 0 <= new_pos[0] < self.m and 0 <= new_pos[1] < self.n and self.grid[new_pos[0], new_pos[1]] != 'W':
                break

            if self.grid[new_pos[0], new_pos[1]] == 'W':
                self.reward -= 100

            if -1 not in new_pos and self.m not in new_pos and self.n not in new_pos:
               break

        return new_pos

    
    def clean_environment(self):

        while self.dirt_count > 0:

            print(f"Dirt: {self.dirt_count} \n")

            self.grid[self.agent_i, self.agent_j] = '-' # agent moved
            self.agent_i, self.agent_j = self.get_new_pos(self.agent_i, self.agent_j)
            self.reward -= 10

            if self.grid[self.agent_i, self.agent_j] == 'D':
                self.grid[self.agent_i, self.agent_j] == '-' # cleaned
                self.dirt_count -= 1
                self.reward += 500

            

            self.grid[self.agent_i, self.agent_j] = 'A' # new agent location
            

            print(f"Reward: {self.reward}")
            self.display_environment()



In [159]:
env2 = environment(3,3)
env2.create_environment()
env2.display_environment()
env2.clean_environment()

| D   A   -   | 

| -   -   -   | 

| D   W   -   | 


Dirt: 2 

Reward: 90
| D   -   -   | 

| -   A   -   | 

| D   W   -   | 


Dirt: 2 

Reward: 80
| D   -   -   | 

| A   -   -   | 

| D   W   -   | 


Dirt: 2 

Reward: 70
| D   -   -   | 

| -   A   -   | 

| D   W   -   | 


Dirt: 2 

Reward: 60
| D   A   -   | 

| -   -   -   | 

| D   W   -   | 


Dirt: 2 

Reward: 50
| D   -   A   | 

| -   -   -   | 

| D   W   -   | 


Dirt: 2 

Reward: 40
| D   A   -   | 

| -   -   -   | 

| D   W   -   | 


Dirt: 2 

Reward: 530
| A   -   -   | 

| -   -   -   | 

| D   W   -   | 


Dirt: 1 

Reward: 520
| -   A   -   | 

| -   -   -   | 

| D   W   -   | 


Dirt: 1 

Reward: 510
| -   -   -   | 

| -   A   -   | 

| D   W   -   | 


Dirt: 1 

Reward: 500
| -   -   -   | 

| A   -   -   | 

| D   W   -   | 


Dirt: 1 

Reward: 990
| -   -   -   | 

| -   -   -   | 

| A   W   -   | 


